### Transfer Learning
Here you will learn how to use pre-trained networks to solved challenging problems.  Here you will use networks trained on a imagenet available from torchvision.

ImageNet is a massive dataset with over 1 million labelled images in 1000 categories.  Its used to train deep neural networks using an architecture called convulational layers. 

Once trained, these models work astonishingly well as feature detectors for images they werent trained on. 

Using a pre-trained network on images not in the training set is called transfer learning.  Here we'll use transfer learning to train a network learning to train a network that can classify our cat and dog photos with near perfect accuracy. 

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import torch
from torch import nn, optim
import torch.nn.functional as F
import torchvision.models as models
